# EcoHome Energy Advisor - RAG Setup

In this notebook, you'll set up the Retrieval-Augmented Generation (RAG) pipeline for the EcoHome Energy Advisor. This will allow the agent to access and cite relevant energy-saving tips and best practices.

## Learning Objectives
- Set up ChromaDB vector store
- Load and process energy-saving documents
- Create embeddings for document chunks
- Implement semantic search functionality
- Test the RAG pipeline

## Documents Available
- `tip_device_best_practices.txt` - Device-specific optimization tips
- `tip_energy_savings.txt` - General energy-saving strategies


## 1. Import Required Libraries


In [ ]:
# Import the necessary libraries for RAG setup
import os
from langchain_chroma  import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

In [ ]:
load_dotenv()

## 2. Load and Process Documents


In [ ]:
# Load the energy-saving tip documents
# Load both tip_device_best_practices.txt and tip_energy_savings.txt
# Use TextLoader to load the documents

documents = []
document_paths = [
    "data/documents/tip_device_best_practices.txt",
    "data/documents/tip_energy_savings.txt"
]

for doc_path in document_paths:
    if os.path.exists(doc_path):
        loader = TextLoader(doc_path)
        docs = loader.load()
        documents.extend(docs)
        print(f"Loaded {len(docs)} documents from {doc_path}")
    else:
        print(f"Warning: {doc_path} not found")

print(f"Total documents loaded: {len(documents)}")


## 3. Split Documents into Chunks


In [ ]:
# Split documents into smaller chunks for better retrieval
# Use RecursiveCharacterTextSplitter with appropriate chunk_size and chunk_overlap
# Experiment with different chunk sizes (e.g., 500, 1000, 1500 characters)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Split the documents
splits = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(splits)} chunks")

# Show sample chunk
if splits:
    print(f"\nSample chunk (first 200 characters):")
    print(splits[0].page_content[:200] + "...")


## 4. Create Vector Store


In [ ]:
# Create a ChromaDB vector store
# Initialize OpenAIEmbeddings
# Create the vector store with the document chunks
# Persist the vector store to disk for future use

# Set up the persist directory
persist_directory = "data/vectorstore"
os.makedirs(persist_directory, exist_ok=True)

# Initialize embeddings
embeddings = OpenAIEmbeddings(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("VOCAREUM_API_KEY")
)

# Create the vector store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(f"Vector store created and persisted to {persist_directory}")
print(f"Total vectors stored: {len(splits)}")

## 5. Test the RAG Pipeline


In [ ]:
# Test the search functionality
# Try different queries related to energy optimization
# Test queries like:
# - "electric vehicle charging tips"
# - "thermostat optimization"
# - "dishwasher energy saving"
# - "solar power maximization"

test_queries = [
    "electric vehicle charging tips",
    "thermostat optimization",
    "dishwasher energy saving",
    "solar power maximization",
    "HVAC system efficiency",
    "pool pump scheduling"
]

print("=== Testing Vector Search ===")
for query in test_queries:
    print(f"\nQuery: '{query}'")
    docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(docs):
        print(f"  Result {i+1}: {doc.page_content[:100]}...")


## 6. Test the Search Tool


In [ ]:
# Test the search_energy_tips tool from tools.py
# Import and test the tool with various queries
# Verify that it returns relevant results

from tools import search_energy_tips

# Test the search_energy_tips function
print("=== Testing search_energy_tips Tool ===")

test_queries = [
    "electric vehicle charging",
    "thermostat settings",
    "dishwasher optimization",
    "solar power tips"
]

for query in test_queries:
    print(f"\nQuery: '{query}'")
    result = search_energy_tips.invoke(
        input={
            "query": query, 
            "max_results": 3,
        }
    )
    
    if "error" in result:
        print(f"  Error: {result['error']}")
    else:
        print(f"  Found {result['total_results']} results")
        for i, tip in enumerate(result['tips']):
            print(f"    {i+1}. {tip['content'][:100]}...")
            print(f"       Source: {tip['source']}")
            print(f"       Relevance: {tip['relevance_score']}")
